In [1]:
import cv2
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output

In [2]:
import os
path = r'F:\PGDBA\Contest\Janta Hackathon'
os.chdir(path)

## Load Model

In [3]:
net = cv2.dnn.readNet("yolov3_training_last_single_veh_comp_train.weights"
                      ,"yolov3_training_last_single_veh_comp_train.cfg")
classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

## Read Test/Train List

In [4]:
df1 = pd.read_excel('test.xlsx')
df1 = np.array(df1)

## Multiple Images

In [5]:
df1.shape[0]

706

In [6]:
img_path = r'F:\PGDBA\Contest\Janta Hackathon\images_backup'

In [7]:
file_name = []
class_label = []
confidences = []
boxes = []
img_shape = []
# df1.shape[0]
for i in range(0,df1.shape[0]):
    filename = df1[i][0]
    os.chdir(img_path)    
    img = cv2.imread("%s"%filename)
    img = cv2.resize(img, None, fx=1, fy=1)
    height, width, channels = img.shape    
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)    
    net.setInput(blob)
    outs = net.forward(output_layers)
    clear_output()
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                x_cen = int(detection[0] * width)
                y_cen = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                label = str(classes[class_id])                
                file_name.append(filename)
                class_label.append(label)
                confidences.append(float(confidence))
                boxes.append([x_cen, y_cen, w, h])
                img_shape.append([width, height])                
                print(i,'-',df1.shape[0],'-',filename)

705 - 706 - 998.jpg


In [12]:
file_name = pd.DataFrame(file_name)
file_name.rename(columns={0:'image_names'},inplace=True)
class_label = pd.DataFrame(class_label)
class_label.rename(columns={0:'class'},inplace=True)
confidences = pd.DataFrame(confidences)
confidences.rename(columns={0:'confidences'},inplace=True)
boxes = pd.DataFrame(boxes)
boxes.rename(columns={0:'x_cen', 1:'y_cen', 2:'w', 3:'h'},inplace=True)
img_shape = pd.DataFrame(img_shape)
img_shape.rename(columns={0:'img_w', 1:'img_ht'},inplace=True)

In [13]:
df2 = pd.concat([file_name, class_label, confidences, boxes, img_shape], axis=1)

In [14]:
df2['emergency_or_not'] = np.where(df2['class']=='non-em', '0', '1')
df2.shape

(707, 10)

In [15]:
df2.head()

,image_names,class,confidences,x_cen,y_cen,w,h,img_w,img_ht,emergency_or_not
0,100.jpg,em,0.996092,100,115,204,161,224,224,1
1,1000.jpg,non-em,0.999638,116,117,222,99,224,224,0
2,1001.jpg,non-em,0.999643,111,109,222,193,224,224,0
3,1004.jpg,non-em,0.998603,117,109,221,160,224,224,0
4,1014.jpg,em,0.733051,111,115,218,224,224,224,1


In [16]:
os.chdir(r'F:\PGDBA\Contest\Janta Hackathon')
df2.to_csv('test_output.csv')